### Hotel in Jeju Data
- '부킹닷컴'의 제주 내 호텔 성급, 가격, 위치, 후기를 스크래핑합니다.

In [33]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import InvalidSessionIdException
from time import sleep
import re
import json
import pandas as pd

In [50]:
url = 'https://www.booking.com/searchresults.ko.html?label=gen173nr-1DCAEoggI46AdIM1gEaH2IAQGYARe4AQfIAQzYAQPoAQGIAgGoAgO4Ar3AsYMGwAIB0gIkNjQ0OTNkZTgtOWUzNC00ODBjLTlmZjktYjA1ZjcwOTFmN2Vk2AIE4AIB&sid=febadbcd7e51aee26acbfeeea6e8299e&sb=1&src=searchresults&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.ko.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaH2IAQGYARe4AQfIAQzYAQPoAQGIAgGoAgO4Ar3AsYMGwAIB0gIkNjQ0OTNkZTgtOWUzNC00ODBjLTlmZjktYjA1ZjcwOTFmN2Vk2AIE4AIB%3Bsid%3Dfebadbcd7e51aee26acbfeeea6e8299e%3Btmpl%3Dsearchresults%3Bclass_interval%3D1%3Bdest_id%3D4170%3Bdest_type%3Dregion%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dregion%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dindex%3Bsrc_elem%3Dsb%3Bsrpvid%3Dadca5de230db0091%3Bss%3D%25EC%25A0%259C%25EC%25A3%25BC%25EB%258F%2584%252C%2520%25EB%258C%2580%25ED%2595%259C%25EB%25AF%25BC%25EA%25B5%25AD%3Bss_all%3D0%3Bssb%3Dempty%3Bsshis%3D0%3Btop_ufis%3D1%26%3B&ss=%EC%A0%9C%EC%A3%BC%EB%8F%84&is_ski_area=0&ssne=%EC%A0%9C%EC%A3%BC%EB%8F%84&ssne_untouched=%EC%A0%9C%EC%A3%BC%EB%8F%84&dest_id=4170&dest_type=region&checkin_year=2021&checkin_month=4&checkin_monthday=24&checkout_year=2021&checkout_month=4&checkout_monthday=25&group_adults=2&group_children=0&no_rooms=1&from_sf=1'
path = '/Users/seyoung/2021/dss17/eda_etc/chromedriver'
# path_f = '/Users/seyoung/2021/dss17/eda_etc/geckodriver'
driver = webdriver.Chrome(path)
# driver = webdriver.Firefox(executable_path=path_f)
driver.get(url)

In [51]:
page_bar = driver.find_elements_by_css_selector('#search_results_table > div.bui-pagination.results-paging_simplified.js-results-paging > nav > ul > li.bui-pagination__pages > ul')[0]
page_bar

<selenium.webdriver.remote.webelement.WebElement (session="dbec807ebbd2f536b0bb1e53ee1b483a", element="fe1b7b44-0ef8-40d7-9bb9-04eebf233d44")>

In [52]:
# 전체 페이지 개수 확인
try:
    pages = page_bar.find_elements_by_css_selector('a > div.bui-u-inline')
    last_pages = ''

    for idx, page in enumerate(pages):
        print(page.text)
        if idx == len(pages)-1: # 마지막 페이지
            last_pages = page.text
except:
    pages = 0
    last_pages = 0    

1
2
3
4
5
19


In [53]:
# 현재 페이지 번호
def get_curr():
    try:
        curr_page = driver.find_elements_by_css_selector('#search_results_table > div.bui-pagination.results-paging_simplified.js-results-paging > nav > ul > li.bui-pagination__pages > ul > li.bui-pagination__item.bui-pagination__item--active.sr_pagination_item.current > a > div.bui-u-inline')[0].text
    except:
        curr_page = 0
    return int(curr_page)

In [54]:
# 성급 (성급이 표시되지 않은 호텔 제외)
def get_stars(listed):
    stars = []

    for i in range(1, len(listed)):
        try:
            like = driver.find_element_by_xpath('//*[@id="hotellist_inner"]/div['+ str(i) +']/div[2]/div[1]/div[1]/div[1]/span/span[1]/span/span')
            a = like.get_attribute('aria-label')
            stars.append(a[0])
            
        # 성급이 표시되지 않는 호텔 
        except Exception as NoSuchElementException:
            stars.append('')
    
    return stars

In [55]:
# 가격
def get_prices(listed):
    prices = []
    
    for i in range(1, len(listed)):
        try:
            price = listed[0].find_elements_by_xpath('//*[@id="hotellist_inner"]/div['+ str(i) +']/div[2]/div[2]/div/div/div/div/div[2]/div[1]/div[2]/div/div')[0]
            base_price = listed[0].find_elements_by_xpath('//*[@id="hotellist_inner"]/div['+ str(i) +']/div[2]/div[2]/div/div/div/div/div[2]/div[1]/div[2]/div/span')[0]
            
            # strong label로 표시된 가격 포함
            if base_price.text == '':
                base_price = listed[0].find_elements_by_xpath('//*[@id="hotellist_inner"]/div[' + str(i) + ']/div[2]/div[2]/div/table/tbody/tr/td[2]/div[2]/strong/label')[0]
            
            prices.append(base_price.text)
            
        # 중간 베너 제외
        except Exception as IndexError:
            prices.append('')
            
    return prices

In [56]:
# 평점
def get_grades(listed):
    grades = []
    
    for i in range(1, len(listed)):
        try:
            grade = listed[0].find_elements_by_xpath('//*[@id="hotellist_inner"]/div[' + str(i) + ']/div[2]/div[1]/div[2]/div/div/a/div/div[1]')[0]
            grades.append(grade.text)
            
        except Exception as IndexError:
            grades.append('')
            
    return grades

In [57]:
# 호텔명
def get_names(listed):
    names = []
    
    for i in range(1, len(listed)):
        try:
            name = listed[0].find_elements_by_xpath('//*[@id="hotellist_inner"]/div['+ str(i) +']/div[2]/div[1]/div[1]/div[1]/h3/a/span[1]')[0]
            names.append(name.text)
            
        # 중간 베너 제외
        except Exception as IndexError:
            names.append('')
            
    return names

In [73]:
# 이용객 후기
def get_reviews():
    good_reviews, bad_reviews, res = [], [], []
    good_review, bad_review = '', ''
    review = driver.find_elements_by_xpath('//*[@id="review_list_page_container"]/ul/li')
    
    for i in range(len(review)):
        
        try:
            good_review += driver.find_elements_by_xpath('//*[@id="review_list_page_container"]/ul/li['+ str(i) +']/div/div[2]/div[2]/div[2]/div/div[1]/p/span[3]')[0].text +'\n'
        
        except Exception as IndexError:
            good_review += ''
            
        try:
            bad_review += driver.find_elements_by_xpath('//*[@id="review_list_page_container"]/ul/li['+ str(i) +']/div/div[2]/div[2]/div[2]/div/div[2]/p/span[3]')[0].text +'\n'
            
        except Exception as IndexError:
            bad_review += ''
        
    good_reviews.append(good_review)
    bad_reviews.append(bad_review)
    
    res.append(good_reviews)
    res.append(bad_reviews)
    
    return res

In [59]:
# 주소
def get_addrs():
    
    addr = driver.find_elements_by_xpath('//*[@id="showMap2"]/span[1]')[0]

    return addr.text

In [67]:
# 새 탭 열기
def open_newTab(listed):
    
    addrs, res, res_reviews = [], [], []
    
    for i in range(1, len(listed)):

        try:
            href = driver.find_element_by_xpath('//*[@id="hotellist_inner"]/div['+ str(i) +']/div[2]/div[1]/div[1]/div[1]/h3/a')
            
        except Exception as NoSuchElementException:
                addrs.append('')
                res_reviews.append('')
                continue
        else:
            href.send_keys(Keys.CONTROL + '\n') 

           # 새로운 탭으로 이동 (2번째 탭)
            driver.switch_to.window(driver.window_handles[1])  

            res_addrs = get_addrs()
            addrs.append(res_addrs)

            # 이용 후기 메뉴로 이동
            comment_btn = driver.find_element_by_xpath('//*[@id="show_reviews_tab"]')
            driver.execute_script("arguments[0].click();", comment_btn)

            res_reviews = get_reviews()
            res.append(res_reviews)
    
            driver.implicitly_wait(1)
            time.sleep(1+random.uniform(0,1))

            driver.close()
            driver.switch_to_window(driver.window_handles[0])
        
    return list(zip(res, addrs))

In [76]:
def data_scraping():
    res_stars, res_names, res_prices, res_grades, res_addrs, res_goodReviews, res_badReviews = [], [], [], [], [], [], []
    
    root_div = driver.find_elements_by_css_selector('#hotellist_inner')[0]
    # 한 페이지당 호출되는 목록 수
    listed = root_div.find_elements_by_xpath('//*[@id="hotellist_inner"]/div')
    
    res_stars = get_stars(listed)
    res_names = get_names(listed)
    res_prices = get_prices(listed)
    res_grades = get_grades(listed)
    
    res = open_newTab(listed)
    
    for i in range(len(res)):
        res_addrs.append(res[i][1])
        res_goodReviews.append(res[i][0][0])
        res_badReviews.append(res[i][0][1])
    
    return list(zip(res_names, res_stars, res_prices, res_grades, res_addrs, res_goodReviews, res_badReviews))

data_scraping()

[('하이 제주 호텔',
  '3',
  '',
  '7.7',
  '제주, 일주서로 5125',
  ['다음에 또 가고싶은 곳 입니다.\n조식은 없었구요\n방도크고 트윈이라 너무너무너무 좋았어요!!\n간단한 싱크대있어서 과일씻어먹기도 좋구 따신물도 팍팍 아주 잘나왔습니다.\n주변 조용하고 비양도보이는 전망이 최고\n베란다에 빨래널기 편리\n야외수영장 너무 좋아요..직원분들 친절하시고\n방이 넓어요\n가성비 최고\n가성비 좋았습니다~~ 침구류 깔끔했고 관리상태도 좋았습니다 출장으로는 최고~\n방크기 : 네가족이 쓰기에 엄청 큼 그리고 방방마다 다 시스템 에어컨이 있음\n위치 : 협재랑 멀지않아 저녁에도 협재쪽가서 놀고와도 문제 없음\n가격대비 만족도가 굉장히 좋음\n식사외에는 아주 편안하고 즐거웠습니다..\n곰팡이 냄새 ㅠㅠ 에어컨 용량이커서 잘때 너무 추움 ㅠ\n풍경이 좋아요. 바다도 보이고 섬도 보이고.. 풍경은 만족합니다. 그리고 처음 체크인 할때 침구랑 확인 했는데 깨끗해요. 찝찝하거나 가렵거나 그런 느낌 없습니다. 포근해요 냄새도 안나구요. 방 넒어요. 1인용 침대 1개 2인용 침대 한개 있고 케리어 큰거 2개.펼쳤는데요 좁은 느낍 없습니다. 까다롭지.읺은 친구랑 여행한다면 100프로 추천합니다. 자고 있는데.새소리도 들리고 파도 소리도 들려요. 힐링은 완벽하게 됩니다.\n'],
  ['바퀴벌레 엄청큰게 한마리나왔어요^^.....................ㅋ\n장마철이라서 그런지 시트에서 냄새가 나서 교환요청했는데 바로 해주셨구요..\n뭐 딱히 없음\n수영장물이 좀 더러웟네요~\n아무래도 오랜 된곳이라 약간의 곰팡이 냄새가 남\n하지만 그것외엔 가성비 최고임\n코로나 때문에 프론트에 직원이 없을때도 있었지만 전화로 바로 연결이 되었기 때문에 불편한건 없었어요 직원이 많지 않은지, 호텔의 운영 방안인지는 모르겠으나 처음 체크인 할때 청소는 깨끗이 되어 있는데 3박 4일동안 방에 들어와 청소는 안해주더라구요. 수건이나 물병은 방문에 비닐 봉지에 담아 걸

In [79]:
def get_hotelDatas():
    raw_datas = []
    
    # 스크래핑 페이지 범위 (현재 페이지 ~ 마지막 페이지)
    while get_curr() <= int(last_pages):
        # scraping
        result = data_scraping()
        raw_datas.append(result)

        if get_curr() == int(last_pages):
            break

        curr_page = driver.find_elements_by_css_selector('#search_results_table > div.bui-pagination.results-paging_simplified.js-results-paging > nav > ul > li.bui-pagination__pages > ul > li.bui-pagination__item.bui-pagination__item--active.sr_pagination_item.current > a > div.bui-u-inline')[0]
        print(curr_page.text,"페이지 완료")
        
        try:
            move_next = driver.find_elements_by_css_selector('#search_results_table > div.bui-pagination.results-paging_simplified.js-results-paging > nav > ul > li.bui-pagination__item.bui-pagination__next-arrow > a')[0]
            move_next.send_keys('\n')

            driver.implicitly_wait(1)
            time.sleep(1+random.uniform(0,1))
        except:
            break
        
    return raw_datas

In [31]:
# 기존 요금, 현재 요금 데이터 파싱
def price_parsing(datas):
    target_prices = []
    
    for i in range(len(datas[0])):
        target_prices.append(datas[0][i][2])

    prices = [] 

    for i in range(len(target_prices)):
        re_prices = re.findall('\d+', target_prices[i])
        base_curr_prices = [] 

        if len(re_prices) == 0:
            base_curr_prices.append('')
            base_curr_prices.append('')
            
        elif len(re_prices) == 2:
            base_curr_prices.append(re_prices[0] + re_prices[1])
            base_curr_prices.append('')
            
        else:
            base_curr_prices.append(re_prices[0] + re_prices[1])
            base_curr_prices.append(re_prices[2] + re_prices[3])

        prices.append(base_curr_prices)

    return prices

In [84]:
datas = get_hotelDatas()
re_prices = price_parsing(datas)

raw_dicts = {'hotel_name':'', 'star':'', 'base_price':'', 'cur_price':'', 'grade':'', 'good_review':'', 'bad_review':'', 'address':''}
raw_datas = []
hotelName_idx, star_idx, grade_idx, addr_idx, review_idx, = 0, 1, 3, 4, 5

print(datas)
for i in range(len(datas[0])):
    
    raw_dicts['hotel_name'] = datas[0][i][hotelName_idx]
    raw_dicts['star'] = datas[0][i][star_idx]
    raw_dicts['base_price'] = re_prices[i][0]
    raw_dicts['cur_price'] = re_prices[i][1]
    raw_dicts['grade'] = datas[0][i][grade_idx]
    raw_dicts['good_review'] = datas[0][i][review_idx][0]
    raw_dicts['bad_review'] = datas[0][i][review_idx+1][0]
    raw_dicts['address'] = datas[0][i][addr_idx]
    
    raw_datas.append(raw_dicts.copy())
    
raw_datas

2 페이지 완료
[[('성산 전망좋은 게스트하우스', '', '요금 ₩40,000', '9.3', '서귀포, 성산읍 일출로 248', ['Thank you so much Summi! I missed saying goodbye to you before leaving.\nTake care of your self! 감사합니다🙏❣️\n멀리보이는 바다뷰가 아닌 가까이에 있어\n아주 만족스러웠습니다.\n광치기해변과 섭지코지가 이어져\n산책하기 좋다고 주인님 알려주셔\n그야말로 답답한 일상에서 벗어나\n조용히 힐링하고 싶을때 다시찾고\n싶습니다.\n3층휴게실에서 마시는 커피한잔은\n전망좋은커피숍으로 값을 지불해야할듯.......\n참 행복한 시간이였습니다.\n편안하게 맞아주시고 이야기꽃을 피워주신 주인님!\n저에게로 와 꽃이 되었습니다.\n정성스레 차려주신 조식도 너무 맛있었고, 성산일출봉이 보이는 뷰도 너무 좋았어요!\n거리도 가까워서 다음에 또 방문하고싶어요!!! 친절한 안내와 세심하게 신경써쥬셔서 감사합니다.\n잠자리도 편하게 잘 잤습니다!\n2년전에도 오고\n작년엔 만석이라 다른곳 갔지만 밤에 놀러가서 사장님부부하고 도란도란 떠들었네요\n빨래돌리고나서 걷는거 까먹었는데 하나하나 걷어서 접어서 자리에 올려놔주시기도 하고 ㅜㅜ\n여러가지 이야기 나누는데 정말 진심이 담기도록 의사소통을 해주시는게 가장 좋아요\n숙소에서 보이는 성산일출봉은 당연히 잘 보이구요\n새벽에 숙소 뒤 바닷가 내려가면 서울에서 못봤던 별들 보고 놀랬어요~\n코로나로 힘들지만 오래오래 있었으면 좋겠어요\n저희는 숙소가 너무 좋아서 추가로 장기 숙박을 했는데요. 숙소는 너무 깔끔하고 쾌적합니다. 사장님들도 너무 친절하시고 웃음이 많으신 분들입니다. 숙소 위치나 교통은 바다도 가깝고 성산일출봉도 가깝고, 숙소 앞에 201번 버스나 동부권 관광지를 지나가는 버스 정류장도 있어서 렌트카 없이 교통도 편리해 만족스러웠습니다. 무엇보다 조식으로 매일 아침마다 정성껏 만들어주시는 토스트, 새벽부터 구워주시고 끓

[{'hotel_name': '성산 전망좋은 게스트하우스',
  'star': '',
  'base_price': '40000',
  'cur_price': '',
  'grade': '9.3',
  'good_review': 'Thank you so much Summi! I missed saying goodbye to you before leaving.\nTake care of your self! 감사합니다🙏❣️\n멀리보이는 바다뷰가 아닌 가까이에 있어\n아주 만족스러웠습니다.\n광치기해변과 섭지코지가 이어져\n산책하기 좋다고 주인님 알려주셔\n그야말로 답답한 일상에서 벗어나\n조용히 힐링하고 싶을때 다시찾고\n싶습니다.\n3층휴게실에서 마시는 커피한잔은\n전망좋은커피숍으로 값을 지불해야할듯.......\n참 행복한 시간이였습니다.\n편안하게 맞아주시고 이야기꽃을 피워주신 주인님!\n저에게로 와 꽃이 되었습니다.\n정성스레 차려주신 조식도 너무 맛있었고, 성산일출봉이 보이는 뷰도 너무 좋았어요!\n거리도 가까워서 다음에 또 방문하고싶어요!!! 친절한 안내와 세심하게 신경써쥬셔서 감사합니다.\n잠자리도 편하게 잘 잤습니다!\n2년전에도 오고\n작년엔 만석이라 다른곳 갔지만 밤에 놀러가서 사장님부부하고 도란도란 떠들었네요\n빨래돌리고나서 걷는거 까먹었는데 하나하나 걷어서 접어서 자리에 올려놔주시기도 하고 ㅜㅜ\n여러가지 이야기 나누는데 정말 진심이 담기도록 의사소통을 해주시는게 가장 좋아요\n숙소에서 보이는 성산일출봉은 당연히 잘 보이구요\n새벽에 숙소 뒤 바닷가 내려가면 서울에서 못봤던 별들 보고 놀랬어요~\n코로나로 힘들지만 오래오래 있었으면 좋겠어요\n저희는 숙소가 너무 좋아서 추가로 장기 숙박을 했는데요. 숙소는 너무 깔끔하고 쾌적합니다. 사장님들도 너무 친절하시고 웃음이 많으신 분들입니다. 숙소 위치나 교통은 바다도 가깝고 성산일출봉도 가깝고, 숙소 앞에 201번 버스나 동부권 관광지를 지나가는 버스 정류장도 있어서 렌트카 없이 교통도 편리해 만족스러웠

In [85]:
def get_fields(raw_datas):

    name = [raw_datas[i]['hotel_name'] for i in range(len(raw_datas))]
    star = [raw_datas[i]['star'] for i in range(len(raw_datas))]
    basePrice = [raw_datas[i]['base_price'] for i in range(len(raw_datas))]
    currentPrice = [raw_datas[i]['cur_price'] for i in range(len(raw_datas))]
    grade = [raw_datas[i]['grade'] for i in range(len(raw_datas))]
    good_review = [raw_datas[i]['good_review'] for i in range(len(raw_datas))]
    bad_review = [raw_datas[i]['bad_review'] for i in range(len(raw_datas))]
    address = [raw_datas[i]['address'] for i in range(len(raw_datas))]
    
    data_pd = pd.DataFrame({'name': name, 'star': star, 'basePrice': basePrice, 'currentPrice': currentPrice, 'grade': grade, 'goodReview': good_review, 'badReview': bad_review, 'address': address })
    
    return data_pd

res_jeju_hotel = get_fields(raw_datas)
res_jeju_hotel

,name,star,basePrice,currentPrice,grade,goodReview,badReview,address
0,성산 전망좋은 게스트하우스,,40000,,9.3,Thank you so much Summi! I missed saying goodb...,수압은 올리고 싶었습니다.\n정말 아쉬운점이 없는 숙소였어요~\n체크인시간 쪼금만 ...,"서귀포, 성산읍 일출로 248"
1,유앤아이 게스트하우스,,50049,40000,9.0,위치가좋다. 일하시는 어르신분들이 시크한 것 같으면서 친절하고 서포티브하시다. 깔끔...,지세히보면 매우 깔끔하지만 그냥 대충 보면 뭔가 찝찝한 느낌의 방과 화장실\n없음\...,"제주, 광양8길 1-2"
2,레프트핸더 게스트하우스,,80000,78400,9.4,조용한 세화마을 끝자락에 위치한 조용한 숙소였어요. 주인 부부께서 아주 친절하게 대...,식당까지 조금 멀어요 \n산책 겸 다녀오실 거라면 문제되진 않아요\n차량 이동이라면...,"제주, 구좌읍 상도북2길 17-12"


In [86]:
res_jeju_hotel.to_csv("jeju_hotel_datas.csv", encoding='utf-8-sig', header=False, mode='a') # 이어서 저장

---